In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras import regularizers
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix
from PyFunctions import Functions as func
from PyFunctions import var
from PyFunctions import Viz
from keras.applications.vgg16 import VGG16, preprocess_input

Using plaidml.keras.backend backend.


In [ ]:
def get_vgg16(dim):
    model = Sequential()
    baseModel = VGG16(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    headModel = model.add(AveragePooling2D(pool_size=(7, 7)))
    headModel = model.add(Flatten(name="flatten"))
    headModel = model.add(Dense(128, activation="relu"))
    headModel = model.add(Dropout(0.3))
    headModel = model.add(Dense(3, activation="softmax", name = 'Output'))

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)


    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model


In [ ]:
x_train, x_test, y_train, y_test = func.get_tts('vgg16')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


augment = False 

early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=18)
model_checkpoint = ModelCheckpoint('../models/VGG16-ModelCheckpointWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 350
batch_size = 32
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 15, width_shift_range = .1, height_shift_range = .1, 
                                                           horizontal_flip = True, fill_mode = 'nearest')
    augmentation.fit(x_train)
    vgg_16_history = vgg_16.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, steps_per_epoch = 8,
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    vgg16_history = vgg_16.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


### Loss and Accuracy

In [ ]:
Viz.plot_loss_accuracy(vgg16_history)

### ROC & AUC

In [ ]:
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)
vgg_16.load_weights('../models/VGG16-ModelCheckpointWeights.h5')
Viz.plot_roc_auc(vgg_16, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 1)
else: 
    dim = (var.vgg_dimension[0], var.vgg_dimension[1], 3)
    
vgg_16 = get_vgg16(dim =dim)
vgg_16.load_weights('../models/VGG16-ModelCheckpointWeights.h5')





y_test_prob = [np.argmax(i) for i in vgg_16.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in vgg_16.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])
